In [1]:
import json
import pandas as pd
import sqlalchemy
from dotenv import dotenv_values
from dotenv import load_dotenv
import psycopg2


In [2]:
pip install SQLAlchemy


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install psycopg2-binary


  Using cached psycopg2_binary-2.9.9-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.4 kB)
Using cached psycopg2_binary-2.9.9-cp310-cp310-macosx_11_0_arm64.whl (2.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# break down attributes columns

data = []

with open('data/yelp_academic_dataset_business.json') as f:

    for line in f:
        json_dict = json.loads(line)

        if json_dict['attributes']:

            if 'BusinessParking' in json_dict['attributes'].keys():

                json_dict['attributes']['BusinessParking'] = eval(json_dict['attributes']['BusinessParking'])

        data.append(json_dict)

In [7]:
businesses = pd.json_normalize(data)

In [8]:
# create function for filtering by state

def state_filter(dataframe, column, state):
    businesses_new = dataframe[dataframe[column] == state].copy()
    return businesses_new

In [9]:
businesses_pa = state_filter(businesses, 'state', 'PA')

In [10]:
# drop 
businesses_pa.dropna(subset='categories', inplace=True)
businesses_pa.reset_index(drop=True, inplace=True)

In [11]:
# import category labels from csv and filter dataframe to keep row that have category label


cat_data = pd.read_csv('data/categories.csv', header=None)

categories = cat_data[0].tolist()

businesses_pa_df = pd.DataFrame()

for index, item in businesses_pa['categories'].items():

    for category in categories:
        if category in item:
            businesses_pa_df = pd.concat([businesses_pa_df, businesses_pa[index:index+1]])
            break

In [12]:
businesses_pa_id = businesses_pa_df['business_id'].tolist()

In [23]:
data_review = []

with open('data/yelp_academic_dataset_review.json') as r:

    for line in r:
        json_dict = json.loads(line)

        if json_dict['business_id']:

            for id in businesses_pa_id: 

                if id in json_dict['business_id']:

                    data_review.append(json_dict)

In [24]:
data_review

[{'review_id': 'KU_O5udG6zpxOg-VcAEodg',
  'user_id': 'mh_-eMZ6K5RLWhZyISBhwA',
  'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw',
  'stars': 3.0,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.",
  'date': '2018-07-07 22:09:11'},
 {'review_id': 'AqPFMleE6RsU23_auESxiA',
  'user_id': '_7bHUi9Uuf5__HHc_Q8guQ',
  'business_id': 'kxX2SOes4o-D3ZQBkiMRfA',
  'stars': 5.0,
  'useful': 1,
  'funny': 0,
  'cool': 1,
  'text': "Wow!  Yummy, different,  delicious.   Our favorite is the lamb c

In [26]:
review_pa = pd.DataFrame(data_review)

In [27]:
review_pa

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
2,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
3,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16
4,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
...,...,...,...,...,...,...,...,...,...
1205557,MVg4YUQeEhCA7Z7RsBJSVg,7-7A0Avj47slLGV7yBFc8w,ytynqOUb3hjKeJfRj5Tshw,3.0,1,0,0,"I was so excited about all the food I saw, but...",2013-07-25 21:00:15
1205558,nLjbVsETpqO17RbFcqskkA,am7-gkH_PDz598oTdYSD6A,3gVSrS4kffGGZT8oXHsIcw,3.0,2,0,2,"*Later Yelp* I've only been here once, but I l...",2014-11-03 14:45:46
1205559,KlHxcAifUF5zDKpJCBrRsw,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4.0,1,0,0,"ice cream! ice cream sodas, sundaes!! \n\nwant...",2012-10-21 04:08:40
1205560,cACxcUY_AIsQKkpDRXuqnw,MCzlzlOw7IGbRAKVjJBPtg,fcGexL5VH5G2Xw0tRj9uOQ,3.0,1,1,0,This is a good pizza option - they deliver thr...,2018-03-13 13:54:48


In [29]:
review_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205562 entries, 0 to 1205561
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   review_id    1205562 non-null  object 
 1   user_id      1205562 non-null  object 
 2   business_id  1205562 non-null  object 
 3   stars        1205562 non-null  float64
 4   useful       1205562 non-null  int64  
 5   funny        1205562 non-null  int64  
 6   cool         1205562 non-null  int64  
 7   text         1205562 non-null  object 
 8   date         1205562 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 82.8+ MB


SQL

In [30]:
import sql_functions as sf
from sql_functions import get_sql_config, get_data, get_dataframe, get_engine

In [31]:
load_dotenv()

True

In [32]:
engine = get_engine()

In [33]:
schema = 'hh_analytics_23_3' # your course schema name, example 'hh_analytics_22_1
table_name = 'group3_review_pa' # Example: 'carriers_pw' for Philipp Wendt

In [34]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try: ## if something goes wrong
        review_pa.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error: 
        print(error)
        engine = None
else:
    print('No engine')

The group3_review_pa table was imported successfully.
